In [1]:
from binance.cm_futures import CMFutures
from binance.um_futures import UMFutures
from binance.spot import Spot
from pandas import DataFrame

spot_client = Spot()
cm_futures_client = CMFutures() # settled in cryptocurrency
um_futures_client = UMFutures() # settled in USDT USDC


In [2]:
# get server time
spot_prices = spot_client.ticker_price()
# create df
spot_prices_df = DataFrame(spot_prices)
spot_prices_df

,symbol,price
0,ETHBTC,0.03568000
1,LTCBTC,0.00106200
2,BNBBTC,0.00765300
3,NEOBTC,0.00015500
4,QTUMETH,0.00096500
...,...,...
2867,USUALBTC,0.00001136
2868,USUALFDUSD,1.07760000
2869,USUALTRY,38.07000000
2870,1000CATUSDC,0.03647000


In [3]:
cm_futures_prices = cm_futures_client.ticker_price()
cm_futures_prices_df = DataFrame(cm_futures_prices)
cm_futures_prices_df
# filter all symbols that end with number
cm_futures_prices_df_n = cm_futures_prices_df[cm_futures_prices_df.symbol.str.contains(r'\d$')]
cm_futures_prices_df_n['symbol'].values
# split all values by usd and leav ethe first part
cm_futures_prices_df_n['symbol'].str.split('USD').str[0].values

array(['LINK', 'DOT', 'XRP', 'ETH', 'XRP', 'ADA', 'LTC', 'ETH', 'BTC',
       'DOT', 'BCH', 'BTC', 'BNB', 'LTC', 'SOL', 'LINK', 'BCH', 'SOL',
       'ADA', 'BNB'], dtype=object)

In [4]:
# um_futures_prices = um_futures_client.ticker_price()
# um_futures_prices_df = DataFrame(um_futures_prices)
# um_futures_prices_df

In [5]:
def cash_and_carry(spot_price, futures_price, interest_rate, days_to_expiry):
    return (futures_price - spot_price) / spot_price * 365 / days_to_expiry - interest_rate

# from cm_futures_prices_df get all tickers containing "BTC"
btc_cm_futures_prices_df = cm_futures_prices_df[cm_futures_prices_df['symbol'].str.contains("BTC")]
# print the row ith the highest price
btc_cm_futures_prices_df_max = btc_cm_futures_prices_df[btc_cm_futures_prices_df['price'] == btc_cm_futures_prices_df['price'].max()]
print(btc_cm_futures_prices_df_max)
print(btc_cm_futures_prices_df_max['symbol'].values[0])
max_price = float(btc_cm_futures_prices_df['price'].max())
btc_spot_prices_df = spot_prices_df[spot_prices_df['symbol'] == "BTCUSDT"]
print(btc_spot_prices_df)
spot_price = float(btc_spot_prices_df['price'].values[0])
# Get the difference between the max price and the spot price percentage
percentage = ((max_price - spot_price) / spot_price) * 100
print(percentage)

           symbol      ps    price           time
18  BTCUSD_250328  BTCUSD  98124.8  1735409291282
BTCUSD_250328
     symbol           price
11  BTCUSDT  95023.79000000
3.263403827609917


In [1]:
def future_premium(cm_futures_prices_df, spot_prices_df, symbol):
    # from cm_futures_prices_df get all tickers containing "BTC"
    btc_cm_futures_prices_df = cm_futures_prices_df[cm_futures_prices_df['symbol'].str.contains(
        symbol)]
    # print the row ith the highest price
    btc_cm_futures_prices_df_max = btc_cm_futures_prices_df[
        btc_cm_futures_prices_df['price'] == btc_cm_futures_prices_df['price'].max()]
    max_price = float(btc_cm_futures_prices_df['price'].max())
    btc_spot_prices_df = spot_prices_df[spot_prices_df['symbol'] == f"{
        symbol}USDT"]
    spot_price = float(btc_spot_prices_df['price'].values[0])
    # Get the difference between the max price and the spot price percentage
    percentage = (((max_price - spot_price) / spot_price) * 100) # - 0.4 # 0.4 is the tx fee
    # rount to 2 decimal places
    return btc_cm_futures_prices_df_max['symbol'].values[0], round(percentage, 2)


cm_futures_prices = cm_futures_client.ticker_price()
cm_futures_prices_df = DataFrame(cm_futures_prices)
cm_futures_prices_df
# filter all symbols that end with number
cm_futures_prices_df_n = cm_futures_prices_df[cm_futures_prices_df.symbol.str.contains(
    r'\d$')]
cm_futures_prices_df_n['symbol'].values
# split all values by usd and leave the first part
symbols = cm_futures_prices_df_n['symbol'].str.split('USD').str[0].values

# create empty dataframe
df_ = DataFrame(columns=['symbol', 'percentage'])
for symbol in symbols:
    s, p = future_premium(cm_futures_prices_df, spot_prices_df, symbol)
    # append to the dataframe
    df_ = df_._append({'symbol': s, 'percentage': p}, ignore_index=True)
    
# sort dataframe by percentage descending
df_ = df_.sort_values(by='percentage', ascending=False)
# remove duplicates
df_ = df_.drop_duplicates()
# reset index
df_ = df_.reset_index(drop=True)
df_
df_.to_markdown()

NameError: name 'cm_futures_client' is not defined